<a href="https://colab.research.google.com/github/Prof-Greipl/wif-640-machine-learning-sem/blob/master/IML_305_Adding_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We add a loss function.

In [1]:
import numpy as np 
from sklearn import metrics

def activation(z, act_func):
    global _activation
    if act_func == 'relu':
       return np.maximum(z, np.zeros(z.shape))
    
    elif act_func == 'sigmoid':
      return 1.0/(1.0 + np.exp( -z ))

    elif act_func == 'linear':
        return z
    else:
        raise Exception('Activation function is not defined.')


def loss(y_true, y_predicted, loss_function='mse'):
   if loss_function == 'mse':
      return metrics.mean_squared_error( y_true, y_predicted)
   else:
      raise Exception('Loss metric is not defined.')

#added
def get_dZ_from_loss(y, y_predicted, metric):
    if metric == 'mse':
        return y_predicted - y
    else:
        raise Exception('Loss metric is not defined.')
#added    
def get_dactivation(A, act_func):
    if act_func == 'relu':
        return np.maximum(np.sign(A), np.zeros(A.shape)) # 1 if backward input >0, 0 otherwise; then diaganolize

    elif act_func == 'sigmoid':
        h = activation(A, 'sigmoid')
        return h *(1-h)

    elif act_func == 'linear':
        return np.ones(A.shape)

    else:
        raise Exception('Activation function is not defined.')
        
class layer:
  def __init__(self,input_dim, output_dim, activation='relu'):    
    self.activation = activation
    self.input_dim = input_dim
    self.output_dim = output_dim # is this needed?? TODO
    if input_dim > 0:
      self.b = np.ones( (output_dim,1) ) 
      self.W = np.ones( (output_dim, input_dim) )
    
    self.A = np.zeros( (output_dim,1) ) # added: we temp. store for A
  
  def setWeight(self, W ):
    self.W = W
    
  def setBias(self, b ):
    self.b = b
    
  def setActivation(self, A ): 
    self.Z =  np.add( np.dot(self.W, A), self.b)
    self.A =  activation(self.Z, self.activation)
  
  
  def print(self, layer_name=""):
    print(f"Konfiguration of Layer {layer_name} ------")
    if self.input_dim > 0:
      print(f"input_dim = {self.input_dim}")
      print(f"output_dim = {self.output_dim}")
      print(f"Activation = {self.activation}")
      print(f"W = ")
      print(self.W)
      print(f"b = ")
      print(self.b)
    else:
      print("This is an input layer..... ")
    #print("-----Finished Layer Config.")
  

class ModelNet:
  def __init__(self, input_dim):  
    
    self.neural_net = []
    self.neural_net.append(layer(0 , input_dim, 'irrelevant'))
    
  def addLayer(self, nr_neurons, activation='relu'):    
    layer_index = len(self.neural_net)
    input_dim = self.neural_net[layer_index - 1].output_dim
    new_layer = layer( input_dim, nr_neurons, activation)
    self.neural_net.append( new_layer)
    
  
  def forward_propagation(self, input_vec ):
    self.neural_net[0].A = input_vec
    for layer_index in range(1,len(self.neural_net)):    
      _A_Prev = self.neural_net[layer_index-1].A                       
      self.neural_net[layer_index].setActivation( _A_Prev )
    return  self.neural_net[layer_index].A
    
    
  # added 
  def backward_propagation(self, y, y_predicted, num_train_datum, metric='mse', verbose=False):
    nr_layers = len(self.neural_net)
    for layer_index in range(nr_layers-1,0,-1):
        if layer_index+1 == nr_layers: # if output layer
            dZ = get_dZ_from_loss(y, y_predicted, metric)
        else: 
            dZ = np.multiply(
                   np.dot(
                       self.neural_net[layer_index+1].W.T, 
                       dZ), 
                   get_dactivation(
                         self.neural_net[layer_index].A, 
                         self.neural_net[layer_index].activation)
                   )
           
        
        dW = np.dot(dZ, self.neural_net[layer_index-1].A.T) / num_train_datum
        db = np.sum(dZ, axis=1, keepdims=True) / num_train_datum
        
        self.neural_net[layer_index].dW = dW
        self.neural_net[layer_index].db = db
        if verbose:
          print(f"\n\n====== Backward Propagation Layer {layer_index} =======")
          print(f"dZ      =  {dZ}");          
          print(f"dW      =  {dW}");
          print(f"\nb     =  {db}");
             
  # added
  def update( self, learning_rate ):
    nr_layers = len(self.neural_net)
    for layer_index in range(1,nr_layers):        # update (W,b)
      self.neural_net[layer_index].W = self.neural_net[layer_index].W - learning_rate * self.neural_net[layer_index].dW  
      self.neural_net[layer_index].b = self.neural_net[layer_index].b - learning_rate * self.neural_net[layer_index].db

  def summary(self):      
      for layer_index in range(len(self.neural_net)):        
        self.neural_net[layer_index].print(layer_index)
        
#Testing        
input_dim = 2
output_dim = 1
model = ModelNet( input_dim )
model.addLayer( 2, 'relu' )
model.addLayer( output_dim, 'linear' )


X  = np.array( [[1,2], [1,2]] ) 
y_true =np.array( [[2, 3]] )

y_predicted = model.forward_propagation( X )
print(f" Predicted value {y_predicted}")
model.backward_propagation(y_true, y_predicted, 1, verbose=True)
model.update( 0.1 )
model.summary()

y_predicted = model.forward_propagation( X )
print(f" Predicted value after one update (=learning) cycle {y_predicted}")



 Predicted value [[ 7. 11.]]


====== Backward Propagation Layer 2 =======
dZ      =  [[5. 8.]]
dW      =  [[55. 55.]]

b     =  [[13.]]


====== Backward Propagation Layer 1 =======
dZ      =  [[5. 8.]
 [5. 8.]]
dW      =  [[21. 21.]
 [21. 21.]]

b     =  [[13.]
 [13.]]
Konfiguration of Layer 0 ------
This is an input layer..... 
Konfiguration of Layer 1 ------
input_dim = 2
output_dim = 2
Activation = relu
W = 
[[-1.1 -1.1]
 [-1.1 -1.1]]
b = 
[[-0.3]
 [-0.3]]
Konfiguration of Layer 2 ------
input_dim = 2
output_dim = 1
Activation = linear
W = 
[[-4.5 -4.5]]
b = 
[[-0.3]]
 Predicted value after one update (=learning) cycle [[-0.3 -0.3]]
